<a href="https://colab.research.google.com/github/askewai/git/blob/main/misiRahasia_C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

MODEL

In [1]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from skimage.feature import graycomatrix, graycoprops
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cross_decomposition import PLSRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import random

# 1. Fungsi Preprocessing & Segmentasi
def white_balance(img):
    result = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(result)
    l = cv2.equalizeHist(l)
    result = cv2.merge((l, a, b))
    return cv2.cvtColor(result, cv2.COLOR_LAB2BGR)

def segment_green_hsv(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    lower_green = np.array([25, 40, 40])
    upper_green = np.array([90, 255, 255])
    mask = cv2.inRange(hsv, lower_green, upper_green)
    return mask

# 2. Fungsi Ekstraksi Fitur Warna & Indeks Vegetasi
def extract_color_features(img, mask):
    masked_img = cv2.bitwise_and(img, img, mask=mask)
    mean_rgb = cv2.mean(masked_img, mask=mask)[:3]
    mean_hsv = cv2.mean(cv2.cvtColor(masked_img, cv2.COLOR_BGR2HSV), mask=mask)[:3]
    mean_lab = cv2.mean(cv2.cvtColor(masked_img, cv2.COLOR_BGR2LAB), mask=mask)[:3]
    return mean_rgb + mean_hsv + mean_lab

def extract_vegetation_indices(img):
    b, g, r = cv2.split(img)
    exg = 2 * g - r - b
    ndvi = (g.astype(float) - r.astype(float)) / (g + r + 1e-5)
    return np.mean(exg), np.mean(ndvi)

# 3. Fungsi Estimasi Pigmen
def predict_chlorophyll(exg, ndvi):
    return 5.65 * ndvi + 3.42 * exg + 1.25

def predict_carotenoid(ndvi, exg):
    return 4.21 * ndvi + 2.93 * exg + 0.85

def predict_anthocyanin(lab_l, hsv_h):
    return 3.78 * lab_l - 1.45 * hsv_h + 0.92

# 4. Membaca Dataset & Menjalankan Model
data_dir = "/content/sample_data/bayam"
feature_list = []
image_names = []

for img_name in os.listdir(data_dir):
    img_path = os.path.join(data_dir, img_name)
    img = cv2.imread(img_path)
    img = white_balance(img)
    mask_hsv = segment_green_hsv(img)

    features = extract_color_features(img, mask_hsv) + extract_vegetation_indices(img)
    feature_list.append(features)
    image_names.append(img_name)

# Konversi ke DataFrame
df_features = pd.DataFrame(feature_list, columns=["R", "G", "B", "H", "S", "V", "L", "A", "B_LAB", "ExG", "NDVI"])
df_features.insert(0, "Image_Name", image_names)

# Prediksi Pigmen
df_features["Chlorophyll"] = df_features.apply(lambda row: predict_chlorophyll(row["ExG"], row["NDVI"]), axis=1)
df_features["Carotenoid"] = df_features.apply(lambda row: predict_carotenoid(row["NDVI"], row["ExG"]), axis=1)
df_features["Anthocyanin"] = df_features.apply(lambda row: predict_anthocyanin(row["L"], row["H"]), axis=1)

# 5. Model Prediksi Pigmen
X = df_features.drop(columns=["Image_Name", "Chlorophyll", "Carotenoid", "Anthocyanin"])
y = df_features[["Chlorophyll", "Carotenoid", "Anthocyanin"]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model CNN
cnn_model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(3, activation='linear')  # Output 3 variabel: Chlorophyll, Carotenoid, Anthocyanin
])
cnn_model.compile(optimizer=Adam(), loss='mse')
cnn_model.fit(X_train, y_train, epochs=50, batch_size=8, verbose=1)

# Simpan Model
cnn_model.save("cnn_pigment_model_1.h5")
print("Model CNN telah dilatih dan disimpan.")

# Output Hasil Prediksi
print("Prediksi Pigmen:")
print(df_features[["Image_Name", "Chlorophyll", "Carotenoid", "Anthocyanin"]].head(15))

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 219236272.0000
Epoch 2/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 247205184.0000 
Epoch 3/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 221572432.0000 
Epoch 4/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 230083072.0000
Epoch 5/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 211615680.0000
Epoch 6/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 234050976.0000 
Epoch 7/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 198327296.0000 
Epoch 8/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 199071408.0000 
Epoch 9/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 175768640.0000
Epoch 10/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 171725184.0000  
Epoch 11/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 189797056.0000 
Epoch 12/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 175814384.0000 
Epoch 13/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 170604432.0000 
Epoch 14/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 156905616.000

Model CNN telah dilatih dan disimpan.
Prediksi Pigmen:
   Image_Name   Chlorophyll    Carotenoid  Anthocyanin
0   B0025.jpg -17559.128364 -13017.162408   725.406240
1   B0001.jpg -22697.938973 -16861.191510   687.491347
2   B0012.jpg  -6857.169045  -5069.215767   689.162480
3   B0027.jpg    338.978084    276.861422   630.117211
4   B0032.jpg  -2836.365596  -2087.181335   627.627038
5   B0017.jpg  29021.468754  21653.546659   583.100014
6   B0010.jpg  21788.621882  16271.208999   513.338278
7   B0006.jpg -22521.778084 -16706.902745   643.428230
8   B0011.jpg  19297.483456  14411.656385   555.086958
9   B0030.jpg  11858.708103   8860.556947   595.964216
10  B0039.jpg  26268.742161  19615.490478   575.375154
11  B0035.jpg  21582.090344  16106.505464   586.519944
12  B0003.jpg  -9791.561549  -7267.005086   633.935551
13  B0026.jpg  -4250.308796  -3089.226334   615.848522
14  B0007.jpg  32679.267032  24388.001047   503.569818


MODEL TEST

In [2]:
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanSquaredError

# Load model yang sudah disimpan
model = load_model("cnn_pigment_model_1.h5", custom_objects={"mse": MeanSquaredError()})

# Fungsi Preprocessing & Segmentasi
def white_balance(img):
    result = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(result)
    l = cv2.equalizeHist(l)
    result = cv2.merge((l, a, b))
    return cv2.cvtColor(result, cv2.COLOR_LAB2BGR)

def segment_green_hsv(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    lower_green = np.array([25, 40, 40])
    upper_green = np.array([90, 255, 255])
    mask = cv2.inRange(hsv, lower_green, upper_green)
    return mask

# Ekstraksi Fitur dari Citra
def extract_color_features(img, mask):
    masked_img = cv2.bitwise_and(img, img, mask=mask)
    mean_rgb = cv2.mean(masked_img, mask=mask)[:3]
    mean_hsv = cv2.mean(cv2.cvtColor(masked_img, cv2.COLOR_BGR2HSV), mask=mask)[:3]
    mean_lab = cv2.mean(cv2.cvtColor(masked_img, cv2.COLOR_BGR2LAB), mask=mask)[:3]
    return mean_rgb + mean_hsv + mean_lab

def extract_vegetation_indices(img):
    b, g, r = cv2.split(img)
    exg = 2 * g - r - b
    ndvi = (g.astype(float) - r.astype(float)) / (g + r + 1e-5)
    return np.mean(exg), np.mean(ndvi)

# Fungsi Prediksi Pigmen dari Citra
def predict_pigment(image_path):
    img = cv2.imread(image_path)
    img = white_balance(img)
    mask_hsv = segment_green_hsv(img)

    features = extract_color_features(img, mask_hsv) + extract_vegetation_indices(img)
    features = np.array(features).reshape(1, -1)  # Ubah ke bentuk yang sesuai untuk model

    prediction = model.predict(features)[0]  # Model memprediksi 3 nilai (Chlorophyll, Carotenoid, Anthocyanin)
    chlorophyll, carotenoid, anthocyanin = prediction

    # Menentukan status kesehatan berdasarkan ketiga pigmen
    if chlorophyll > 4.5 and carotenoid > 2.0 and anthocyanin < 1.5:
        status = "Sehat"
    elif 3.0 < chlorophyll <= 4.5 or 1.0 < carotenoid <= 2.0 or 1.5 <= anthocyanin <= 2.5:
        status = "Stres"
    else:
        status = "Tidak Sehat"

    return chlorophyll, carotenoid, anthocyanin, status

# Test prediksi
test_image = "/content/test_B_1.jpg"
chlorophyll_value, carotenoid_value, anthocyanin_value, health_status = predict_pigment(test_image)

print(f"Prediksi Klorofil: {chlorophyll_value:.2f}")
print(f"Prediksi Karotenoid: {carotenoid_value:.2f}")
print(f"Prediksi Antosianin: {anthocyanin_value:.2f}")
print(f"Status Kesehatan: {health_status}")

# Test prediksi
test_image = "/content/test_B_2.jpg"
chlorophyll_value, carotenoid_value, anthocyanin_value, health_status = predict_pigment(test_image)

print(f"Prediksi Klorofil: {chlorophyll_value:.2f}")
print(f"Prediksi Karotenoid: {carotenoid_value:.2f}")
print(f"Prediksi Antosianin: {anthocyanin_value:.2f}")
print(f"Status Kesehatan: {health_status}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 225ms/step
Prediksi Klorofil: -31739.06
Prediksi Karotenoid: -26798.37
Prediksi Antosianin: 1283.86
Status Kesehatan: Tidak Sehat
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Prediksi Klorofil: -29857.56
Prediksi Karotenoid: -25217.13
Prediksi Antosianin: 1209.55
Status Kesehatan: Tidak Sehat


========================================